In [1]:
%%writefile cmm_cuda.cu

#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <stdio.h>
#include <stdlib.h>
#include <climits>

// Helper function to calculate diagonal-based index
__device__ int getDiagonalIndex(int i, int j, int n) {
    int diagonal = j - i;
    int position = i - 1;

    int offset = 0;
    for (int d = 0; d < diagonal; d++) {
        offset += (n - d);
    }

    return offset + position;
}

// Serial CPU implementation
void chainMatrixMultiplication_Serial(int* dim, int** M, int** A, int n) {
    for (int i = 1; i <= n; i++) {
        M[i][i] = 0;
    }

    for (int len = 2; len <= n; len++) {
        for (int i = 1; i <= n - len + 1; i++) {
            int j = i + len - 1;
            M[i][j] = INT_MAX;

            for (int k = i; k <= j - 1; k++) {
                int cost = M[i][k] + M[k + 1][j] + dim[i - 1] * dim[k] * dim[j];
                if (cost < M[i][j]) {
                    M[i][j] = cost;
                    A[i][j] = k;
                }
            }
        }
    }
}

// Basic CUDA kernel
__global__ void CMM_CUDA_kernel(int* dev_dim, int* dev_m, int* dev_result, int n) {
    int tid = threadIdx.x;
    int matrix_size = n + 1;

    if (tid < n) {
        dev_m[tid * matrix_size + tid] = 0;
    }
    __syncthreads();

    for (int len = 2; len <= n; len++) {
        int diagonal_length = n - len + 1;

        if (tid < diagonal_length) {
            int i = tid + 1;
            int j = i + len - 1;

            int min_cost = INT_MAX;
            for (int k = i; k <= j - 1; k++) {
                int cost = dev_m[i * matrix_size + k] +
                          dev_m[(k + 1) * matrix_size + j] +
                          dev_dim[i - 1] * dev_dim[k] * dev_dim[j];
                if (cost < min_cost) {
                    min_cost = cost;
                }
            }
            dev_m[i * matrix_size + j] = min_cost;
        }
        __syncthreads();
    }

    if (tid == 0) {
        *dev_result = dev_m[1 * matrix_size + n];
    }
}

// Optimized CUDA kernel
__global__ void CMM_CUDA_optimized_kernel(int* dev_dim, int* dev_m_diagonal, int* dev_result, int n) {
    int tid = threadIdx.x;

    if (tid < n) {
        int diagonal_idx = tid;
        dev_m_diagonal[diagonal_idx] = 0;
    }
    __syncthreads();

    for (int len = 2; len <= n; len++) {
        int diagonal_length = n - len + 1;

        if (tid < diagonal_length) {
            int i = tid + 1;
            int j = i + len - 1;

            int min_cost = INT_MAX;
            for (int k = i; k <= j - 1; k++) {
                int left_diag_idx = getDiagonalIndex(i, k, n);
                int right_diag_idx = getDiagonalIndex(k + 1, j, n);

                int cost = dev_m_diagonal[left_diag_idx] +
                          dev_m_diagonal[right_diag_idx] +
                          dev_dim[i - 1] * dev_dim[k] * dev_dim[j];

                if (cost < min_cost) {
                    min_cost = cost;
                }
            }

            int current_diag_idx = getDiagonalIndex(i, j, n);
            dev_m_diagonal[current_diag_idx] = min_cost;
        }
        __syncthreads();
    }

    if (tid == 0) {
        int final_idx = getDiagonalIndex(1, n, n);
        *dev_result = dev_m_diagonal[final_idx];
    }
}

// Function to run single test
void runSingleTest(int n, float* serial_time, float* cuda_time, float* optimized_time) {
    // Initialize dimensions
    int* dim = (int*)malloc((n + 1) * sizeof(int));
    srand(42); // Fixed seed for consistent results
    for (int i = 0; i <= n; i++) {
        dim[i] = rand() % 100 + 1;
    }

    // Host memory allocation
    int** M = (int**)malloc((n + 1) * sizeof(int*));
    int** A = (int**)malloc((n + 1) * sizeof(int*));
    for (int i = 0; i <= n; i++) {
        M[i] = (int*)malloc((n + 1) * sizeof(int));
        A[i] = (int*)malloc((n + 1) * sizeof(int));
    }

    // Device memory allocation
    int* dev_dim;
    int* dev_m;
    int* dev_m_diagonal;
    int* dev_result;

    int matrix_size = (n + 1) * (n + 1);
    int diagonal_size = (n * (n + 1)) / 2;

    cudaMalloc(&dev_dim, (n + 1) * sizeof(int));
    cudaMalloc(&dev_m, matrix_size * sizeof(int));
    cudaMalloc(&dev_m_diagonal, diagonal_size * sizeof(int));
    cudaMalloc(&dev_result, sizeof(int));

    cudaMemcpy(dev_dim, dim, (n + 1) * sizeof(int), cudaMemcpyHostToDevice);

    // CUDA Events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Timing serial version
    cudaEventRecord(start);
    chainMatrixMultiplication_Serial(dim, M, A, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(serial_time, start, stop);

    // Timing basic CUDA version
    cudaEventRecord(start);
    CMM_CUDA_kernel<<<1, n>>>(dev_dim, dev_m, dev_result, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(cuda_time, start, stop);

    // Timing optimized CUDA version
    cudaEventRecord(start);
    CMM_CUDA_optimized_kernel<<<1, n>>>(dev_dim, dev_m_diagonal, dev_result, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(optimized_time, start, stop);

    // Cleanup
    cudaFree(dev_dim);
    cudaFree(dev_m);
    cudaFree(dev_m_diagonal);
    cudaFree(dev_result);

    for (int i = 0; i <= n; i++) {
        free(M[i]);
        free(A[i]);
    }
    free(M);
    free(A);
    free(dim);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    // Reset GPU state to avoid interference
    cudaDeviceReset();
}

int main() {
    printf("Chain Matrix Multiplication Performance Comparison\n");
    printf("================================================\n");
    printf("| n    | Serial (ms) | CUDA (ms) | Optimized (ms) | CUDA Speedup | Opt Speedup |\n");
    printf("|------|-------------|-----------|----------------|--------------|-------------|\n");

    // Test range from 1016 to 1024
    for (int n = 1016; n <= 1024; n++) {
        float serial_time, cuda_time, optimized_time;

        printf("Testing n = %d... ", n);
        fflush(stdout);

        runSingleTest(n, &serial_time, &cuda_time, &optimized_time);

        float cuda_speedup = serial_time / cuda_time;
        float opt_speedup = serial_time / optimized_time;

        printf("Done\n");
        printf("| %4d | %11.2f | %9.2f | %14.2f | %12.2fx | %11.2fx |\n",
               n, serial_time, cuda_time, optimized_time, cuda_speedup, opt_speedup);
    }

    printf("\nPerformance Analysis:\n");
    printf("- Serial time grows as O(n³)\n");
    printf("- CUDA basic should show consistent performance\n");
    printf("- Optimized version performance depends on memory access patterns\n");
    printf("- Each test uses fresh GPU state to avoid interference\n");

    return 0;
}

// Alternative: Multiple runs with averaging
void runMultipleTests(int n, int num_runs, float* avg_serial, float* avg_cuda, float* avg_optimized) {
    float total_serial = 0, total_cuda = 0, total_optimized = 0;

    for (int run = 0; run < num_runs; run++) {
        float serial_time, cuda_time, optimized_time;
        runSingleTest(n, &serial_time, &cuda_time, &optimized_time);

        total_serial += serial_time;
        total_cuda += cuda_time;
        total_optimized += optimized_time;
    }

    *avg_serial = total_serial / num_runs;
    *avg_cuda = total_cuda / num_runs;
    *avg_optimized = total_optimized / num_runs;
}

Writing cmm_cuda.cu


In [2]:
!nvcc -arch=sm_75 cmm_cuda.cu -o cmm_cuda

In [3]:
!./cmm_cuda

Chain Matrix Multiplication Performance Comparison
| n    | Serial (ms) | CUDA (ms) | Optimized (ms) | CUDA Speedup | Opt Speedup |
|------|-------------|-----------|----------------|--------------|-------------|
Testing n = 1016... Done
| 1016 |     1442.25 |    350.61 |        1236.31 |         4.11x |        1.17x |
Testing n = 1017... Done
| 1017 |     2134.02 |    329.12 |        1235.78 |         6.48x |        1.73x |
Testing n = 1018... Done
| 1018 |     1471.09 |    175.22 |        1235.52 |         8.40x |        1.19x |
Testing n = 1019... Done
| 1019 |     1480.34 |    170.28 |        1239.59 |         8.69x |        1.19x |
Testing n = 1020... Done
| 1020 |     1535.18 |    168.26 |        1243.61 |         9.12x |        1.23x |
Testing n = 1021... Done
| 1021 |     1799.60 |    334.65 |        1247.89 |         5.38x |        1.44x |
Testing n = 1022... Done
| 1022 |     1455.19 |    261.08 |        1251.93 |         5.57x |        1.16x |
Testing n = 1023... Done
| 1023